In [1]:
import requests
from requests import get
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

In [2]:
def flatten_dict(df, column_name):
    df1 = df.copy()
    flatten_col = json_normalize(df1[column_name].tolist())
    df1 = pd.concat([df1,flatten_col], sort = False, axis = 1)
    df1 = df1.drop(column_name, axis = 1)
    return df1

Read the API here...

https://www.mediawiki.org/wiki/API:Revisions

In [3]:
BASE_URL = "http://en.wikipedia.org/w/api.php"
TITLE = 'Donald Trump'

parameters = { 'action': 'query',
           'format': 'json',
           'continue': '',
           'titles': TITLE,
           'prop': 'revisions',
           'rvprop': 'ids|userid|user|timestamp|comment|content|tags',
           'rvlimit': 'max'}

wp_call = requests.get(BASE_URL, params=parameters)
response = wp_call.json()

In [4]:
total_revisions = 0
pages = []
counter = 0
num_calls = 6
while True:
    wp_call = requests.get(BASE_URL, params=parameters)
    response = wp_call.json()
    
    val = str(json_normalize(response,["query","pages"])[0][0])
    df = json_normalize(response,["query","pages",val,"revisions"])
    pages.append(df)

    for page_id in response['query']['pages']:
        total_revisions += len(response['query']['pages'][page_id]['revisions'])

    if 'continue' in response:
        parameters['continue'] = response['continue']['continue']
        parameters['rvcontinue'] = response['continue']['rvcontinue']
        counter += 1
    else:
        break
    if counter == num_calls:
        break

In [5]:
revisions = pd.concat(pages, sort = False)

In [7]:
revisions.head()

,*,comment,contentformat,contentmodel,parentid,revid,tags,timestamp,user,userid
0,{{pp-vand|small=yes}}\n{{pp-move-indef|small=y...,/* External links */,text/x-wiki,wikitext,893341599,893390112,[],2019-04-21T00:47:46Z,Rreagan007,6320553
1,{{pp-vand|small=yes}}\n{{pp-move-indef|small=y...,/* Relationship with the press */,text/x-wiki,wikitext,893341337,893341599,[],2019-04-20T18:21:32Z,Rreagan007,6320553
2,{{pp-vand|small=yes}}\n{{pp-move-indef|small=y...,/* Sexual misconduct allegations */,text/x-wiki,wikitext,893340892,893341337,[],2019-04-20T18:18:59Z,Rreagan007,6320553
3,{{pp-vand|small=yes}}\n{{pp-move-indef|small=y...,/* Special counsel */,text/x-wiki,wikitext,893340591,893340892,[],2019-04-20T18:14:54Z,Rreagan007,6320553
4,{{pp-vand|small=yes}}\n{{pp-move-indef|small=y...,/* Russian interference */,text/x-wiki,wikitext,893334303,893340591,[],2019-04-20T18:12:16Z,Rreagan007,6320553
